In [24]:
import numpy as np

############################# initialize parameter ####################################
dic = {'PRESTART':0,'START':1,'B-positive':2,'I-positive':3,'B-neutral':4,'I-neutral':5,'B-negative':6,'I-negative':7,
       'O':8,'STOP':9,'POSTSTOP':10}
l = ['PRESTART', 'START', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 
     'I-negative', 'O', 'STOP', 'POSTSTOP']

# store emission parameters
# data structure: tuple + dictionary
e_param = ({}, {}, {}, {}, {}, {}, {}, {}, {})  ## 1st,2nd dict empty
obs_space = set()

# store transition parameters
# initialize as a 11*11*11 matrix of zeros
a = 11
t_param = np.zeros((11,11,11))

# count # of sentiment appears+1
count = [1] * a

def forward(preScore, x, preState, prepreState): #!!!!!!!!!!!!!!!!!!!!!!!! need help, doesn't really know how to do :(
    """inputs: preScore: list of (pre_score real_num, pre_parent int)
               x: current word
       output: list of max(score real_num, parent int) for all states, len=7
    """
    layer = []
    for i in range(2, 9):  # i: 2~8
        temp_score = []
        # calculate emission first
        if ((x in obs_space) & (x in e_param[i])):
            b = e_param[i][x] / count[i]
        elif (x in obs_space):
            b = 0 / count[i] #!!!!!!!!!!!!!!!!!!!!!!!!!!
        else:
            b = 1.0 / count[i]
        for j in range(1, 4):  # j:1-7
            # score = preScore*a*b
            j_score = preScore[j-1][0] * (t_param[j][i]) * b  # trans 1~7 -> 1-7  ?????????????????????????don't know how to do
            temp_score.append(j_score)
        max_value = max(temp_score)
        max_index = temp_score.index(max_value)  # index: 0-6
        layer.append((max_value, max_index))
    return layer


def viterbiAlgo(X):
    # initialization
    n = len(X)
    Y = []
    prev_layer = []
    # prestart, start -> 1
    x = X[0]
    for j in range(2, 9):
        if ((x in obs_space) & (x in e_param[j])):
            b = e_param[j][x] / count[i]
        elif (x in obs_space):
            b = 0 / count[i]  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        else:
            b = 1.0 / count[j]
        prob = t_param[0][1][j] * b
        prev_layer.append((prob, 0))  # (prob, START)
    layers = [[(1,-1)],prev_layer]
    preState = 'PRESTART'
    prepreState = 'START'

    # calculate path i=(1,...,n)
    for i in range(1, n):  # 1 -> n-1
        score = forward(layers[i], X[i], preState, prepreState)  # a list of max(score: real, parent: int) for all 7 states
        layers.append(score)
        prepreState = preState
        preState = X[i]

    # calculate score(n+1, STOP), and get max
    temp_score = []
    for j in range(1, 4):
        # score = preScore*a
        t_score = layers[n][j-1][0] * (t_param[j][4])
        temp_score.append(t_score)
    max_value = max(temp_score)
    max_index = temp_score.index(max_value)
    layers.append([(max_value, max_index)])
    # print(scores)

    # backtracking
    parent = 0  # only 1 entry in STOP
    for i in range(n+1, 1, -1):  # index range from N to 2
        parent = layers[i][parent][1]
        Y.insert(0, l[parent + 1])  # 1-7
    #print(Y)
    return Y

def updateParam(X, Y, Ytrain):
    for i in range(2,len(Y)):
        t_param[dic[Y[i-2]]][dic[Y[i-1]]][dic[Y[i]]] += 1
        t_param[dic[Ytrain[i-2]]][dic[Ytrain[i-1]]][dic[Ytrain[i]]] -= 1
    for i in range(2, len(Y)-2):
        temp_dic = e_param[dic[Y[i]]]
        if (X[i-2] in temp_dic):
            temp_dic[X[i-2]] += 1
        elif (X[i-2] in obs_space):
            temp_dic[X[i-2]] = 1
        else:
            temp_dic[X[i-2]] = 1
            obs_space.add(X[i-2])
    for i in range(2, len(Y)-2):
        temp_dic = e_param[dic[Ytrain[i]]]
        if (X[i-2] in temp_dic):
            temp_dic[X[i-2]] -= 1
        elif (X[i-2] in obs_space):
            temp_dic[X[i-2]] = -1
        else:
            temp_dic[X[i-2]] = -1
            obs_space.add(X[i-2])      

def train():
    ############################# initialize parameter ####################################
    # num of iteration over the training set
    T = 1

    ## read and parse file
    train_file = open('train_EN', 'r')
    Y = ['PRESTART','START']
    X = []
    
    for trainStep in range(0,T):
        for obs in train_file:
            try:
                obs, v = obs.split()
                obs = obs.strip().lower()
                v = v.strip()
                X.append(obs)
                Y.append(v)
            except:
                # meaning the end of a sentence: x->STOP
                Y.append(['STOP','POSTSTOP'])
                Ytrain = viterbiAlgo(X)
                
                # reset
                Y = ['PRESTART','START']
                X = []

#train()
#viterbiAlgo(['New', 'Year',','])
updateParam(['New', 'Year',',','New'], ['PRESTART','START','B-positive','B-positive','O','O','STOP', 'POSTSTOP'], ['PRESTART','START','O','O','O','O','STOP', 'POSTSTOP'])
print(e_param)
print(obs_space)
print(t_param)

({}, {}, {'New': 1, 'Year': 1}, {}, {}, {}, {}, {}, {',': 0, 'New': -1, 'Year': -1})
{',', 'New', 'Year'}
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  1. ..., -1.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  1. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ..., -1.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  1.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  1.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ..., -2.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0